In [1]:
!pip install opencv-python-headless matplotlib pyyaml torch torchvision clip-by-openai pytorch-lightning
!pip install -U ultralytics


INFO: pip is looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
ERROR: Cannot install clip-by-openai==0.1.0, clip-by-openai==0.1.1, clip-by-openai==0.1.1.2, clip-by-openai==0.1.1.3, clip-by-openai==0.1.1.4, clip-by-openai==0.1.1.5, clip-by-openai==1.0.1, clip-by-openai==1.1, torch and torchvision==0.12.0 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch
    torchvision 0.12.0 depends on torch==1.11.0
    clip-by-openai 1.1 depends on torch<1

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import cv2
import numpy as np
import torch
import yaml
from PIL import Image
import matplotlib.pyplot as plt
from ultralytics import YOLO
from torchvision import transforms
from torch.nn import functional as F


In [3]:
!pip install ultralytics


In [4]:
from ultralytics import YOLO
import cv2
import os


In [5]:
# Load YOLOv8n (nano) model
yolo = YOLO('yolov8n.pt')  # Pretrained YOLOv8 Nano model


In [6]:
from google.colab import files
uploaded = files.upload()


Saving 105.mp4 to 105.mp4


In [9]:
# Perform object detection on the video
results = yolo("/content/105.mp4")

# Ensure the detections directory exists
save_dir = 'detections/'
os.makedirs(save_dir, exist_ok=True)

# Iterate over each result and save
for i, result in enumerate(results):
    save_path = os.path.join(save_dir, f"frame_{i}.jpg")  # Save each frame's detection
    result.plot()  # Plot the result on the frame
    result.save(save_path)  # Save the result frame to the path




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/96) /content/105.mp4: 384x640 4 persons, 9.4ms
video 1/1 (frame 2/96) /content/105.mp4: 384x640 4 persons, 9.2ms
video 1/1 (frame 3/96) /content/105.mp4: 384x640 4 persons, 8.0ms
video 1/1 (frame 4/96) /content/105.mp4: 384x640 4 persons, 9.6ms
video 1/1 (frame 5/96) /content/105.mp4: 384x640 3 persons, 7.1ms
video 1/1 (frame 6/96) /content/105.mp4: 384x640 3 persons, 1 chair, 11.1ms
video 1/1 (frame 7/96) /content/105.mp4: 384x640 3

In [11]:
for i, result in enumerate(results):
    print(f"Frame {i}: {result.boxes.data}")  # Print bounding box data


Frame 0: tensor([[7.8334e+02, 4.9968e+02, 1.1229e+03, 1.0751e+03, 9.2867e-01, 0.0000e+00],
        [1.2896e+03, 2.5909e+02, 1.5375e+03, 7.1457e+02, 8.1476e-01, 0.0000e+00],
        [1.3392e+03, 3.6991e+02, 1.6773e+03, 9.0189e+02, 8.0074e-01, 0.0000e+00],
        [1.5542e+03, 9.2145e+02, 1.8389e+03, 1.0767e+03, 4.3688e-01, 0.0000e+00]], device='cuda:0')
Frame 1: tensor([[7.7589e+02, 4.9211e+02, 1.1120e+03, 1.0740e+03, 9.1620e-01, 0.0000e+00],
        [1.2854e+03, 2.5945e+02, 1.5369e+03, 7.1353e+02, 8.4649e-01, 0.0000e+00],
        [1.3459e+03, 3.7373e+02, 1.6664e+03, 9.0486e+02, 8.3589e-01, 0.0000e+00],
        [1.5443e+03, 9.2010e+02, 1.8373e+03, 1.0759e+03, 3.5318e-01, 0.0000e+00]], device='cuda:0')
Frame 2: tensor([[7.7555e+02, 4.8578e+02, 1.1005e+03, 1.0754e+03, 9.1403e-01, 0.0000e+00],
        [1.2785e+03, 2.5780e+02, 1.5350e+03, 7.0707e+02, 8.6488e-01, 0.0000e+00],
        [1.3597e+03, 3.8571e+02, 1.6746e+03, 9.1664e+02, 7.8027e-01, 0.0000e+00],
        [1.5355e+03, 9.1910e+02, 1.

In [12]:
import numpy as np

# Example: Simple rule-based approach
for i, result in enumerate(results):
    detected_classes = [yolo.names[int(cls)] for cls in result.boxes.cls]
    if 'person' in detected_classes and 'knife' in detected_classes:
        print(f"Potential violence detected in Frame {i}")


In [15]:
import cv2
import os

# Directory where the detection frames are saved
save_dir = 'detections/'

# Get the list of saved frames, sorted to maintain the correct order
frame_files = sorted([os.path.join(save_dir, f) for f in os.listdir(save_dir) if f.endswith('.jpg')])

# Check if the list of frames is not empty
if len(frame_files) == 0:
    print("No frames found in the detections directory. Please check the path and ensure frames are saved.")
else:
    # Get the frame size from the first frame
    sample_frame = cv2.imread(frame_files[0])
    frame_size = sample_frame.shape[1::-1]  # (width, height)

    # Initialize the video writer
    output_video = 'output_results.mp4'
    video_writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), 30, frame_size)

    # Iterate over frames and write them to the video
    for frame_file in frame_files:
        frame = cv2.imread(frame_file)
        video_writer.write(frame)  #


In [16]:
from IPython.display import HTML
from base64 import b64encode

def display_video(video_path):
    with open(video_path, "rb") as f:
        video = f.read()
    video_encoded = b64encode(video).decode("ascii")
    return HTML(f"""
    <video width="600" controls>
        <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
    </video>
    """)

# Display the video
display_video(output_video)


Output hidden; open in https://colab.research.google.com to view.

In [18]:
import cv2
import os
from ultralytics import YOLO

# Load the YOLO model
yolo = YOLO('yolov8n.pt')  # You can choose a model that fits your needs

# Directory where the detection frames are saved
save_dir = 'detections/'

# Get the list of saved frames, sorted to maintain the correct order
frame_files = sorted([os.path.join(save_dir, f) for f in os.listdir(save_dir) if f.endswith('.jpg')])

# Check if the list of frames is not empty
if len(frame_files) == 0:
    print("No frames found in the detections directory. Please check the path and ensure frames are saved.")
else:
    # Get the frame size from the first frame
    sample_frame = cv2.imread(frame_files[0])
    frame_size = sample_frame.shape[1::-1]  # (width, height)

    # Initialize the video writer
    output_video = 'output_results.mp4'
    video_writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), 30, frame_size)

    # Iterate over frames and write them to the video
    for frame_file in frame_files:
        frame = cv2.imread(frame_file)

        # Perform object detection on the current frame
        results = yolo(frame)

        # Extract bounding box data and labels for detected objects
        detected_violence = False  # Default to No Violence Detected
        for result in results:
            for box in result.boxes:
                # Here, you can define which objects/labels indicate violence
                label = box.cls  # Get the class label for each detected object
                if label == 'weapon' or label == 'aggressive behavior':  # Example for detecting violence
                    detected_violence = True
                    break

        # Add the detection text to the frame
        detection_text = "Violence Detected" if detected_violence else "No Violence Detected"
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, detection_text, (50, 50), font, 1, (0, 0, 255), 2, cv2.LINE_AA)  # Red text

        # Write the frame to the video
        video_writer.write(frame)

    video_writer.release()  # Save the video
    print(f"Video saved as {output_video}")



0: 384x640 2 persons, 17.3ms
Speed: 4.3ms preprocess, 17.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6.5ms
Speed: 3.0ms preprocess, 6.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 6.8ms
Speed: 3.5ms preprocess, 6.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 handbag, 15.8ms
Speed: 3.4ms preprocess, 15.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 7.1ms
Speed: 3.4ms preprocess, 7.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 handbag, 7.2ms
Speed: 3.3ms preprocess, 7.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 7.3ms
Speed: 3.4ms preprocess, 7.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 7.2ms
Speed: 3.6ms preprocess, 7.2ms inference, 1.2ms postprocess per image 

In [20]:
from IPython.display import HTML
from base64 import b64encode

def display_video(video_path):
    with open(video_path, "rb") as f:
        video = f.read()
    video_encoded = b64encode(video).decode("ascii")
    return HTML(f"""
    <video width="600" controls>
        <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
    </video>
    """)

# Display the video
display_video(output_video)


Output hidden; open in https://colab.research.google.com to view.

In [21]:
from google.colab import files

# Specify the output video file path
output_video = 'output_results.mp4'

# Download the video file to your local machine
files.download(output_video)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>